# Algorithme génétique

## Les algorithmes évolutifs

### Algorithmes génétiques

Le Machine Learning peut apparaître complexe de prime abord, et pour cause certains des algorithmes qui constituent cet ensemble sont très loin d’être accessibles.

Ceci étant, il existe un type d’algorithme évolutif plus simple à saisir parmi ceux-ci: les algorithmes génétiques (ou AG).

### Principes

Comme leur nom l'indique, les AGs ont certains points communs avec la génétique en biologie.

Un individu est un ensemble de données concaténées, chacune de ses données représente un gène.

L'ensemble des individus représente la population, tandis que chacune des populations (une par étape de l'algorithme est une génération.

D'une génération à l'autre, l'individu le plus apte à répondre à la problématique est conservé tandis que les autres sont obtenus via le croisement des individus de la génération précédente. Au terme de ces croisements, des modifications peuvent avoir lieu: les mutations.

### Une particularité

L'utilisation d'un AG n'est pas, en soi, très compliqué. Ainsi, nous allons ajouter un peu d'éléments relatifs au Big Data et à ses principes dans ces travaux.

Le principe est finalement simple. Le Big Data, et le traitement de données qui l'accompagne sont définis autour de trois grandes étapes qui sont, pour rappel, le Data Cleaning, le Data Mining et enfin le Data Analysis. A chaque nouvelle étape, un nouveau niveau de détail, plus précis, doit être étudié.

Nous verrons dans le déroulé de ce TD les points importants analogues à ces trois grandes étapes de traitement des données.

## Comment faire?

Au final, les AGs ne sont pas particulièrement compliqués à implémenter, et ce TD vous permettra de créer un exemple simple.

**Attention: Le contenu que vous produirez ici sera réutilisé lors du TD Examen AG, qui sera noté. Veillez à la modularité de votre code.**

**Egalement, pensez à commenter votre code et à le rendre lisible (voir la norme PEP8)**

### Objectif

A la fin de ce TD, votre code sera capable de trouver, de lui-même, une chaîne de caractères définie. Bien sûr, l'utilité est faible, mais il s'agit de pouvoir traiter ensuite toute sorte de données, comme par exemple, utiliser l'AG pour estimer une courbe à partir d'un ensemble de points fournis.

Pour ce TD, le code devra retrouver la chaîne de caractères "Hello World".

### Description

Pour fonctionner, l'AG a besoin:
- d'une fonction d'évaluation permettant d'indiquer la proximité ou l'erreur entre l'individu et la solution optimale. Dans notre cas, la distance entre chaque caractère sera utilisée, la fonction d'évaluation pour une distance optimale vaudra donc 0.
- d'une fonction de croisement qui permet d'obtenir une nouvelle génération à partir d'une génération pré-existante, prenant une partie de deux individus pour en former un nouveau, et ce pour l'ensemble de la population.
- de paramètres de mutation, indiquant les chances pour un nouvel individu d'être altéré, et la force de cette altération.

### Avant le code

In [7]:
# tous les imports de ce TD devront être placés ici
from math import dist
import random
import time
import numpy as np
import string

#change une liste asctii en mot
def ascii_to_lettre(ascii_liste):
    return ''.join(chr(i) for i in ascii_liste)

ModuleNotFoundError: No module named 'numpy'

### Création des méthodes

#### Calcul de distance et localisation du meilleur individu

Avant toute chose, il faut réfléchir à la manière de représenter nos données.

Nous traiterons des chaînes de caractères. En Python, une chaîne de caractères fonctionne (presque) exactement comme une liste de caractères, et ces caractères peuvent être remplacés par leur code ASCII, qui est numérique.

Comme dit précédemment, nous allons définir notre individu cible comme la liste des codes ASCII de "H", "e", "l", "l", ... "d", qui aura comme distance 0.

Puisque nous avons les codes ASCII des caractères à disposition, il est simple de trouver, pour chaque indice de la liste, la distance entre le caractère cible et le caractère courant, qui n'est rien de plus qu'une simple différence. De même, la distance globale de la chaîne est une somme des distances.

Une fois les distances d'un ensemble de chaînes obtenues, celles-ci peuvent être ordonnées.

In [11]:
# la fonction de conversion d'une chaîne de caractères en liste de valeurs ASCII vous est founie
def string_to_int_list(string):
    return [ord(character) for character in list(string)]

##### Exo 1:
- **Créez la méthode permettant de calculer la distance entre un mot et un autre**
- **Vérifiez que la distance avec "Hello World" est correcte pour les mots suivants: "COjsy OfUkp" (105) et "Hemlo Wohld" (11)**

In [12]:
# Renvoie la meilleur distance entre 2 mots
def get_distance(string1, string2):
    distance = 0 
    index = 0
    distance_B = 0
    int1 = string_to_int_list(string1)
    int2 = string_to_int_list(string2)
    
    for number in int1:
        distance_B = abs(int1[index] - int2[index])
        distance += distance_B
        index += 1
    
    return distance

Maintenant qu'il est possible d'attribuer une valeur de distance entre deux mots, nous pouvons ordonner des mots grâce à cette valeur.

##### Exo 2:
- **Créez la méthode permettant de retrouver le mot le plus proche d'une cible et la valeur de cette distance**
- **Testez avec une liste de mots définie par vos soins, et la cible "Hello World"**

In [13]:
# Renvoie le meilleur d'une liste et sa distance avec la target
def get_best(string, liste_compare):
    liste_distance = []
    
    for liste_nombre in liste_compare:
        liste_distance.append(get_distance(string,liste_nombre))
        
    min_distance = min(liste_distance)
    mot = liste_compare[liste_distance.index(min_distance)]
    return mot, min_distance

#### Création d'une première génération

Bien que nous connaissions le mot cible, nous partirons d'une population constituée d'un ensemble de mots générés aléatoirement.

Gardez à l'esprit que la taille de votre population définira la vitesse d'exécution de votre code. Ainsi, bien qu'un ensemble d'individus important  aura la diversité pour atteindre rapidement (en termes d'itérations) la solution, un ensemble plus restreint passera chacune des itérations rapidement.

##### Exo 3:
- **Créez la méthode d'initialisation d'une liste de mots aléatoires**
- **Utilisez cette méthode pour générer aléatoirement un ensemble de mots, et comparez-les à la cible.**

In [14]:
# Fonction qui renvoie une liste de mots
# prends en paramêtre le nombre de mot que doit contenir la liste
def word_list_init(nombre_de_mot, lettre):
    liste_mot = []
    
    # genere 5 lettres aléatoire, fait un espace et regènère 5 lettres aléatoire
    # le nombre de fois qui est demandé par la fonction passé en paramètre
    for iteration in range(nombre_de_mot):
        liste_mot.append(''.join(random.choice(string.ascii_letters) 
                                for index in range(lettre)))
    return liste_mot


#### Passage d'une génération à une autre

Les opérations de base sont maintenant établies, il est temps de rendre notre système évolutif.

D'une génération à l'autre, les individus doivent évoluer. Mais, il n'est pas certain que cette évolution se fasse dans la direction espérée.

La théorie de l'évolution dans le domaine biologique indique que l'espèce la plus adaptée à son environnement survivra plus probablement que d'autres. Dans notre cas, nous pouvons discerner l'individu le plus proche de la solution et le conserver (il "survit"). Ainsi, le meilleur individu d'une génération ne peut pas être moins adapté que celui d'une génération passée (principe de non-régression).

Le reste de la population de la nouvelle génération est produite comme dans le cas de la biologie. Deux parties complémentaires (chromosomes) de deux individus (parents) seront combinées pour obtenir un nouvel individu (enfant).

##### Exo 4:
- **Créez la méthode de transition entre deux générations. Notez que:**
 - **Le meilleur individu (ou les x meilleurs individus) devrait être conservé.**
 - **Les individus restants devraient être un croisement d'individus de la liste des mots précédente (meilleur mot précédent compris).**
 - **La liste de mots résultante devrait être de même longueur que la précédente.**
- **Appliquez la méthode de manière itérative, en indiquant à chaque fois le meilleur élément de la génération et la distance avec la cible.**
- **Au bout d'un certain nombre d'itérations, que se passe-t-il?**

In [15]:
# Prends 2 mots en paramêtres
# Renvoie un nouveau mot en mélangeant les 2 mots données
def crossover(string1, string2):
    string1_int = string_to_int_list(string1)
    string2_int = string_to_int_list(string2)
    enfant = []
    string1_length = len(string1_int)
    
    # Il y a 85% de chance de prendre la lettre du premiers mot placé en paramêtre
    for i in range(string1_length):
        choice = np.random.choice(np.arange(0, 2), 
                                  p=[0.85, 0.15])
        if choice == 0: 
            enfant.append(string1_int[i])
        else: 
            enfant.append(string2_int[i])
    
    return enfant


# Génère une nouvelle liste de mots
def new_generation(target, ma_liste):
    new_generation = []
    len_ma_liste = len(ma_liste)
    
    # Ajoute les 5 meilleurs mots au liste new_generation
    # Supprime les 5 meilleur mot de l'ancienne liste
    for i in range(5):
        best_distance = get_best(target, ma_liste)
        best_word = best_distance[0]
        new_generation.append(best_word)
        ma_liste.pop(ma_liste.index(best_word))

    # Tant que la longueur du liste new_generation est différente de la longueur de liste ma_liste
    # Alors random prends un chiffre entre 0 et 4 avec le même % d'avoir chacun des chiffres
    while len(new_generation) != len_ma_liste:
        random = np.random.choice(np.arange(0, 5), p=[0.2, 0.2, 0.2, 0.2, 0.2])
        
        # Si random = 0 alors le crossover se fait avec le meilleur mot
        if best_distance[1] > 100:
            if random == 0:
                random_word1 = new_generation[0]
                random_word2 = new_generation[0]
                new_enfant1 = crossover(random_word1, random_word2)
                new_enfant2 = mutation(new_enfant1,target)
                new_generation.append(ascii_to_lettre(new_enfant2))
                
            # Si random = 1 alors le crossover se fait avec le meilleur mot et le 2e meilleur
            elif random == 1:
                random_word1 = new_generation[0]
                random_word2 = new_generation[1]
                new_enfant1 = crossover(random_word1, random_word2)
                new_enfant2 = mutation(new_enfant1, target)
                new_generation.append(ascii_to_lettre(new_enfant2))
                
            # Si random = 2 alors le crossover se fait avec le 2e meilleur mot et le 3e meilleur
            elif random == 2:
                random_word1 = new_generation[1]
                random_word2 = new_generation[2]
                new_enfant1 = crossover(random_word1, random_word2)
                new_enfant2 = mutation(new_enfant1, target)
                new_generation.append(ascii_to_lettre(new_enfant1))
                
            # Si random = 3 alors le crossover se fait avec le 3e meilleur mot et le 4e meilleur
            elif random == 3:
                random_word1 = new_generation[2]
                random_word2 = new_generation[3]
                new_enfant1 = crossover(random_word1, random_word2)
                new_enfant2 = mutation(new_enfant1, target)
                new_generation.append(ascii_to_lettre(new_enfant1))
                
            # Si random = 4 alors le crossover se fait avec le 4e meilleur mot et le 5e meilleur
            elif random == 4:
                random_word1 = new_generation[3]
                random_word2 = new_generation[4]
                new_enfant1 = crossover(random_word1, random_word2)
                new_enfant2 = mutation(new_enfant1, target)
                new_generation.append(ascii_to_lettre(new_enfant1))
                
        # Sinon le crossover se fait entre un mot parmi les 5 meilleurs et un autre parmi l'ancienne liste
        else:
            random_word2 = np.random.choice(ma_liste)
            random_word1 = np.random.choice(new_generation)
            new_enfant1 = crossover(random_word1, random_word2)
            new_enfant2 = mutation(new_enfant1,target)
            new_generation.append(ascii_to_lettre(new_enfant2))

    return new_generation

Nous retrouvons, en fin de compte, un ensemble d'individus très similaires. A la différence de l'ensemble initial, où tous les éléments sont générés aléatoirement, ce point des travaux montre une grande défaillance en termes de diversité.

S'il fallait considérer le lien avec les grandes phases du Big Data, nous pourrions imaginer un travail de Data Cleaning trop poussé, retirant tous les éléments aberrants mais aussi malheureusement certains éléments essentiels à la complétude espérée de la donnée (dans notre cas, l'accès à un nombre suffisant de lettres par emplacement).

#### Créer la diversité génétique

Le principe de mutation permet d'ajouter de la diversité lors de l'évolution de l'AG. En effet, avec une faible population, il est presque certain que le croisement normal d'éléments finisse dans une impasse, on parle de minimum local.

Dans notre cas, la notion de minimum local n'a pas véritable lieu d'être. Il nous suffit d'ajouter de nouveaux éléments innovants pour permettre à l'AG de reprendre son évolution.

Cette innovation prend la forme d'une altération d'un (ou plusieurs) caractère(s) d'un individu lors de sa création pour une nouvelle génération. Pour s'assurer que l'on ne dégrade pas le meilleur individu, il est conseillé de lui éviter cette étape.

La mutation est définie suivant deux paramètres principaux. Tout d'abord, sa probabilité, entre 0 et 1, définit sa fréquence. Aussi, sa force définit l'effet de la mutation, et peut prendre n'importe quelle forme, de +1/-1 à une réaffectation aléatoire. Cette mutation peut s'appliquer lettre par lettre, sur le mot entier, ou sur un ensemble de lettres aléatoires.

### Parallèle avec le Big Data

Ici, vous chercherez à optimiser votre algorithme, de sorte qu'il puisse vous fournir une réponse rapidement. Un manque de diversité, ou sa perte trop rapide, pose problème. De même, une avancée trop précise sans "risque" entraîne des longueurs nuisibles à l'efficacité du travail.

Il peut donc être très intéressant de transposer par analogie les thématique du Big Data ici.

#### Data Cleaning

L'ensemble de mots initial est aléatoire, et la probabilité de trouver un mot intéressant est assez faible. De même, le croisement entre ces mots prend plus d'importance, en termes de progression, que la mutation, qui apport un précision ponctuelle.

#### Data Mining

A supposer que l'ensemble des individus est maintenant à la fois restreinte à des mots moins aléatoires tout en conservant une diversité importante, il est maintenant possible de se concentrer sur une base plus limitée de mots et tenter de les croiser en appliquant des mutations de puissance moyenne.

#### Data Analysis

Maintenant que l'ensemble d'individus est dans un espace de recherche très petit, le croisement va commencer à perdre en efficacité (les mots sont presque tous identiques). Il est temps d'éliminer les doublons et de concentrer le travail de précision en ne conservant que la mutation, avec les paramètres les plus faibles possibles.

##### Exo 5:
- **Créez une méthode permettant de définir le procédé de mutation**
- **Utilisez cette méthode pour obtenir la terminaison de votre AG (réduire la distance du meilleur mot à 0)**
- **Votre code est complet, faites varier certains de vos paramètres afin de le rendre plus efficace, si vous le souhaitez**
- **Rendez votre code adaptatif, capable d'acception diverses longueurs de chaînes de caractères.**

In [16]:
# Prend en paramêtre un mot et la target
def mutation(mot_ascii, ma_target):
    taille_mot = len(mot_ascii)
    distance = get_distance(ma_target, ascii_to_lettre(mot_ascii))
    
    # Pour chaque caractere du mot donné en paramètre
    for i in range(taille_mot):
        # Si la distance avec la target est entre 100 et 50
        # Alors une chance sur 9 d'effectuer la mutation
        if distance < 100 and distance >= 50:
            chance = random.randint(0,9)
            if chance == 1:
                poids_mutation = distance// 20
                random_number = random.randint(-1 * poids_mutation, poids_mutation)
                mot_ascii[i] = (random_number + mot_ascii[i]) % 255
                
        # Si la distance avec la target est entre 50 et 15
        # Alors une chance sur 12 d'effectuer la mutation
        elif distance < 50 and distance >= 15:
            chance = random.randint(0,12)
            if chance == 1:
                poids_mutation = 2
                random_number = random.randint(-1 * poids_mutation, poids_mutation)
                mot_ascii[i] = (random_number + mot_ascii[i]) % 255
        # Si la distance avec la target est inferieur à 15
        # Alors une chance sur 15 d'effectuer la mutation
        elif distance < 15:
            chance = random.randint(0,15)
            if chance == 1:
                poids_mutation = 1
                random_number = random.randint(-1 * poids_mutation, poids_mutation)
                mot_ascii[i] = (random_number + mot_ascii[i]) % 255
        
        # Sinon 1 chance sur 18 d'effectuer la mutation
        else:
            chance = random.randint(0,18)
            if chance == 1:
                poids_mutation = distance// 10
                random_number = random.randint(-1 * poids_mutation, poids_mutation)
                mot_ascii[i] = (random_number + mot_ascii[i]) % 255 
 
    return mot_ascii

Le bloc suivant vous permet de tester l'exécution de votre code dans les conditions de test finales. Les paramètres des méthodes "word_list_init", "get_distance" et "new_generation" sont à compléter.

La valeur de la variable "test_length" sera modifiée lors de l'évaluation du code.

In [17]:
# Permet de lancer mon algorithme
test_length = 32
target = "".join([chr(random.randint(0, 255)) for _ in range(test_length)])
nb_iteration = 0

def algo_genetique(ma_super_target, nb_iteration):
    print("ma_super_target : ",ma_super_target, "de longueur", len(ma_super_target))

    listeInit = word_list_init(100, len(ma_super_target))

    print("iteration n°" ,nb_iteration)
    new_gene_list = new_generation(ma_super_target, listeInit)
    best_word = get_best(ma_super_target, new_gene_list)[0]
    best_distance = get_best(ma_super_target,new_gene_list)[1]
    print("le meilleur mot est :", best_word,"avec une distance de:",best_distance) 
    # time.sleep(1)

    start = time.time()
    while best_word != ma_super_target:
        # time.sleep(1)
        nb_iteration += 1
        print("iteration n°",ma_super_target)
        new_gene_list = new_generation(ma_super_target, new_gene_list)
        best_word = get_best(ma_super_target, new_gene_list)[0]
        best_distance = get_best(ma_super_target,new_gene_list)[1]
        print("     le meilleur mot est :", best_word,"avec une distance de:",best_distance) 

algo_genetique(target, nb_iteration)

ma_super_target :  òMèãÍÄµ
.Bª±j³Ö_þ¿ð} de longueur 32


NameError: name 'string' is not defined

### Adaptation finale

Maintenant que vous avez un code qui termine, assez efficacement, il est temps de trouver une application plus ludique, mais non moins sérieuse, puisqu'elle sera notée.

Ici, l'objectif sera de retrouver non plus une phrase, mais plutôt une "image". Vous trouverez dans le bloc suivant, la nouvelle cible de recherche, qui est un ascii art. Un exemple ests fourni, vous pouvez parfaitement changer cet ascii art selon vos goûts.

Deux nouvelles contraintes s'ajoutent.

La première, il est nécessaire de reconnaître les caractères problématiques (par exemple, les guillemets ou le caractère d'échappement "\") pour assurer leur lecture.

La seconde, il ne faut pas oublier que l'ascii est définie sur deux dimensions. Une méthode d'affichage, qui demande la chaîne à afficher et la longueur d'une ligne de dessin est fournée.

### Travail demandé

Vous pouvez choisir de réutiliser les méthodes existantes pour retrouver l'ascii art que vous aurez choisi (taille minimale acceptée : 3x3).

In [ ]:
# La cible est :
#/\     /\
#  \ _____\
#   (_)-(_)
ascii_target="/\     /\   \ _____\   (_)-(_)"

def printer_ascii(indiv, length):
    tab_print = [indiv[i:i+length] for i in range(int(len(indiv)/length))]
    for line in tab_print:
        print(line)

In [8]:
# insérez ici votre rendu final
nb_iteration = 0

def en_route_ascii(ma_super_target, nb_iteration):
    # test_length = 32
    # target = "".join([chr(random.randint(0, 255)) for _ in range(test_length)])
    print("target : ",ma_super_target, "de longueur", len(ma_super_target))

    list_init = word_list_init(100, len(ma_super_target))

    print("iteration n°" ,nb_iteration)
    new_gene_list = new_generation(ma_super_target, list_init)
    best_word = get_best(ma_super_target, new_gene_list)[0]
    best_distance = get_best(ma_super_target,new_gene_list)[1]
    print("le meilleur mot est :", best_word,"avec une distance de:",best_distance) 
    # time.sleep(1)

    start = time.time()
    while best_word != ma_super_target:
        # time.sleep(1)
        nb_iteration += 1
        print("iteration n°",nb_iteration)
        new_gene_list = new_generation(ma_super_target, new_gene_list)
        best_word = get_best(ma_super_target, new_gene_list)[0]
        best_distance = get_best(ma_super_target,new_gene_list)[1]
        print("     le meilleur mot est :", best_word,"avec une distance de:",best_distance) 
    print(time.time() - start)
    ascii_int_list = string_to_int_list(best_word)
    target_ascii = printer_ascii(ascii_int_list, 10)
    
    en_route_ascii(ascii_target, nb_iteration)

IMPORTANT:

Ce TD est noté et compte pour un quart de la note TD finale.
L'évaluation sera effectuée par le redémarrage du noyau et l'exécution complète du code. Vérifiez la validité de votre travail avec "Noyau" -> "Redémarrer & tout exécuter". Tout code ne fonctionnant pas en suivant cette procédure vaudra 0.

Le barème est le suivant:
- **L'exécution complète attribuera au plus 12 points.** 2 points sont attribués pour chaque méthode correctement implémentée.
- **Les codes terminant seront mis en compétition** 0 à 6 points seront attribués en fonction du classement.
- **La propreté** (respect du PEP8) **vaudra 2 points.** Un code non propre peut faire perdre jusqu'à 3 points.
- Le code doit respecter le côté aléatoire du sujet. Cela inclut la génération initiale, le croisement et la mutation. Toute méthode parmi les trois indiquées ne respectant pas ce point vaudra 0.

Attention, tout jour de retard pour le rendu de ce travail entraînera une pénalité de 5 points.

Aussi ce code devant être utilisé pour le TD Examen AG, il est conseillé d'y mettre du soin.

Le rendu prend la forme de ce notebook, à envoyer par mail.